# Semantic Kernel với Tích hợp Máy chủ MCP OpenBnB

Notebook này hướng dẫn cách sử dụng Semantic Kernel với máy chủ MCP OpenBnB thực tế để tìm kiếm các chỗ ở Airbnb thực tế bằng MCPStdioPlugin. Để truy cập LLM, nó sử dụng Azure AI Foundry. Để thiết lập các biến môi trường của bạn, bạn có thể làm theo [Bài học Thiết lập](/00-course-setup/README.md)


In [ ]:
# Import cell - Updated imports
import json
import os
import asyncio

from dotenv import load_dotenv
from IPython.display import display, HTML
from typing import Annotated

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.mcp import MCPStdioPlugin
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent, StreamingTextContent

## Tạo Kết Nối Plugin MCP

Chúng ta sẽ kết nối với [máy chủ MCP OpenBnB](https://github.com/openbnb-org/mcp-server-airbnb) bằng cách sử dụng MCPStdioPlugin. Máy chủ này cung cấp chức năng tìm kiếm Airbnb thông qua gói @openbnb/mcp-server-airbnb.


## Tạo Client

Trong ví dụ này, chúng ta sẽ sử dụng Azure AI Foundry để truy cập LLM. Hãy đảm bảo rằng các biến môi trường của bạn đã được thiết lập đúng cách.


## Cấu hình môi trường

Cấu hình các thiết lập Azure OpenAI. Đảm bảo rằng bạn đã thiết lập các biến môi trường sau:
- `AZURE_OPENAI_CHAT_DEPLOYMENT_NAME`
- `AZURE_OPENAI_ENDPOINT`
- `AZURE_OPENAI_API_KEY`


In [ ]:
# Creating the Client cell - Updated for Azure
load_dotenv()

# Azure OpenAI configuration
# Ensure these environment variables are set:
# - AZURE_OPENAI_CHAT_DEPLOYMENT_NAME
# - AZURE_OPENAI_ENDPOINT
# - AZURE_OPENAI_API_KEY (optional if using DefaultAzureCredential)

chat_completion_service = AzureChatCompletion(
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    # Optional - will use DefaultAzureCredential if not set
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
)

## Hiểu về tích hợp OpenBnB MCP

Notebook này kết nối với **máy chủ OpenBnB MCP thực** cung cấp chức năng tìm kiếm Airbnb thực tế.

### Cách hoạt động:

1. **MCPStdioPlugin**: Sử dụng giao tiếp đầu vào/đầu ra tiêu chuẩn với máy chủ MCP  
2. **Gói NPM thực**: Tải xuống và chạy `@openbnb/mcp-server-airbnb` thông qua npx  
3. **Dữ liệu trực tiếp**: Trả về dữ liệu bất động sản Airbnb thực tế từ các API của họ  
4. **Khám phá chức năng**: Agent tự động phát hiện các chức năng có sẵn từ máy chủ MCP  

### Các chức năng có sẵn:

Máy chủ OpenBnB MCP thường cung cấp:  
- **search_listings** - Tìm kiếm bất động sản Airbnb theo địa điểm và tiêu chí  
- **get_listing_details** - Lấy thông tin chi tiết về các bất động sản cụ thể  
- **check_availability** - Kiểm tra tình trạng sẵn có cho các ngày cụ thể  
- **get_reviews** - Lấy đánh giá cho các bất động sản  
- **get_host_info** - Lấy thông tin về chủ sở hữu bất động sản  

### Điều kiện tiên quyết:

- **Node.js** được cài đặt trên hệ thống của bạn  
- **Kết nối Internet** để tải gói máy chủ MCP  
- **NPX** khả dụng (đi kèm với Node.js)  

### Kiểm tra kết nối:

Bạn có thể kiểm tra máy chủ MCP thủ công bằng cách chạy:  
```bash
npx -y @openbnb/mcp-server-airbnb
```  

Lệnh này sẽ tải xuống và khởi động máy chủ OpenBnB MCP, sau đó Semantic Kernel sẽ kết nối để lấy dữ liệu Airbnb thực tế.  


## Chạy Agent với OpenBnB MCP Server

Bây giờ chúng ta sẽ chạy AI Agent kết nối với OpenBnB MCP server để tìm kiếm chỗ ở thực tế trên Airbnb tại Stockholm cho 2 người lớn và 1 trẻ em. Bạn có thể thoải mái thay đổi danh sách `user_inputs` để điều chỉnh tiêu chí tìm kiếm.


In [ ]:
# Main execution cell - Enhanced with proper HTML rendering and MCP tool logging
# User requests for Airbnb search
user_inputs = [
    "Find Airbnb in Stockholm for 2 adults 1 kid",
]


async def main():
    """Main function to run the MCP-enabled agent with real OpenBnB server using Azure OpenAI"""

    try:
        # Create MCP plugin connection to real OpenBnB server
        async with MCPStdioPlugin(
            name="AirbnbSearch",
            description="Search for Airbnb accommodations using OpenBnB MCP server",
            command="npx",
            args=["-y", "@openbnb/mcp-server-airbnb", "--ignore-robots-txt"],
        ) as airbnb_plugin:

            print("🔧 MCP Plugin created and connected")

            # Load tools for function discovery
            await airbnb_plugin.load_tools()
            await asyncio.sleep(3)  # Give more time for initialization
            print("✅ Tools loaded from MCP server")

            # Debug: Check what tools were loaded
            if hasattr(airbnb_plugin, '_tools'):
                print(f"📋 Internal tools: {airbnb_plugin._tools}")

            # Verify available functions
            funcs = [attr for attr in dir(airbnb_plugin)
                     if callable(getattr(airbnb_plugin, attr))
                     and attr in ['airbnb_search', 'airbnb_listing_details']]
            print(f"📋 Available functions: {funcs}")

            # Create agent with Azure OpenAI service
            agent = ChatCompletionAgent(
                service=AzureChatCompletion(),  # Use default constructor
                name="AirbnbAgent",
                instructions="""You are an Airbnb search assistant. Use the airbnb_search function to find properties. 
                Format results in a clear HTML table with columns for property name, price, rating, and link.""",
                plugins=[airbnb_plugin],
            )

            print("🤖 Agent created with Azure OpenAI")

            # Process each user input
            thread: ChatHistoryAgentThread | None = None

            for user_input in user_inputs:
                print(f"\n🔍 Processing request: {user_input}")
                
                # Track MCP tool usage
                mcp_tools_used = []
                function_calls_log = []
                
                # Try streaming to capture function calls
                try:
                    agent_name = None
                    full_response = []
                    current_function_name = None
                    argument_buffer = ""
                    
                    async for response in agent.invoke_stream(
                        messages=user_input,
                        thread=thread,
                    ):
                        thread = response.thread
                        agent_name = response.name
                        
                        for item in response.items:
                            # Log function calls
                            if isinstance(item, FunctionCallContent):
                                if item.function_name:
                                    current_function_name = item.function_name
                                    mcp_tools_used.append(item.function_name)
                                    print(f"\n🔧 MCP Tool Selected: {item.function_name}")
                                    
                                if isinstance(item.arguments, str):
                                    argument_buffer += item.arguments
                            
                            # Log function results
                            elif isinstance(item, FunctionResultContent):
                                if current_function_name:
                                    try:
                                        args = json.loads(argument_buffer.strip()) if argument_buffer else {}
                                    except:
                                        args = {"raw": argument_buffer}
                                    
                                    function_calls_log.append({
                                        "function": current_function_name,
                                        "arguments": args,
                                        "timestamp": asyncio.get_event_loop().time()
                                    })
                                    
                                    print(f"   📍 Arguments: {json.dumps(args, indent=2)}")
                                    print(f"   ✅ MCP Tool Executed Successfully")
                                    
                                    current_function_name = None
                                    argument_buffer = ""
                            
                            # Collect response text
                            elif isinstance(item, StreamingTextContent) and item.text:
                                full_response.append(item.text)
                    
                    # Join the full response
                    response_text = ''.join(full_response)
                    
                except Exception as e:
                    print(f"⚠️ Streaming failed, using get_response: {str(e)[:100]}")
                    # Fallback to non-streaming
                    response = await agent.get_response(messages=user_input, thread=thread)
                    thread = response.thread
                    response_text = str(response)
                    agent_name = response.name
                
                
                # Process the response to ensure HTML tables render correctly
                # Remove any markdown code blocks around HTML
                response_text = response_text.replace('```html', '').replace('```', '')
                
                # Ensure proper HTML structure for tables
                if '<table' in response_text.lower():
                    # Add CSS styling for better table rendering
                    table_css = """
                    <style>
                        .airbnb-results table {
                            border-collapse: collapse;
                            width: 100%;
                            margin: 10px 0;
                        }
                        .airbnb-results th, .airbnb-results td {
                            border: 1px solid #ddd;
                            padding: 8px;
                            text-align: left;
                        }
                        .airbnb-results th {
                            background-color: #f2f2f2;
                            font-weight: bold;
                        }
                        .airbnb-results tr:nth-child(even) {
                            background-color: #f9f9f9;
                        }
                        .airbnb-results a {
                            color: #1976d2;
                            text-decoration: none;
                        }
                        .airbnb-results a:hover {
                            text-decoration: underline;
                        }
                    </style>
                    """
                    response_text = f'{table_css}<div class="airbnb-results">{response_text}</div>'
                
                # Build the complete HTML output
                html_output = f"""
                <div style='margin:10px; padding:10px; border-left:3px solid #2E8B57; background:#F0F8FF;'>
                    <strong>User:</strong> {user_input}
                </div>
                """
                
                # Add function call details if available
                if function_calls_log:
                    details_html = "<details style='margin:10px; padding:10px; background:#f5f5f5;'>"
                    details_html += "<summary><strong>📊 Function Call Details</strong></summary>"
                    details_html += "<pre style='background:#fff; padding:10px; overflow-x:auto;'>"
                    for call in function_calls_log:
                        details_html += f"Function: {call['function']}\n"
                        details_html += f"Arguments: {json.dumps(call['arguments'], indent=2)}\n"
                        details_html += "---\n"
                    details_html += "</pre></details>"
                    html_output += details_html
                
                # Add the agent's response with proper HTML rendering
                html_output += f"""
                <div style='margin:10px; padding:15px; border-left:3px solid #1E90FF; background:#FFFFFF;'>
                    <strong>{agent_name}:</strong><br>
                    {response_text}
                </div>
                """
                
                # Display the HTML with proper rendering
                display(HTML(html_output))
                
                
    except Exception as e:
        print(f"❌ Error: {str(e)}")
        import traceback
        traceback.print_exc()

print("🚀 Starting with Azure OpenAI...")
await main()
print("✅ Done!")

Tóm tắt  
Chúc mừng! Bạn đã thành công xây dựng một tác nhân AI tích hợp với việc tìm kiếm chỗ ở thực tế bằng cách sử dụng Giao thức Ngữ cảnh Mô hình (MCP):  

Công nghệ đã sử dụng:  
- Semantic Kernel - Để xây dựng các tác nhân thông minh với Azure OpenAI  
- Azure AI Foundry - Cho khả năng LLM và hoàn thành hội thoại  
- MCP (Model Context Protocol) - Để tích hợp công cụ theo chuẩn hóa  
- OpenBnB MCP Server - Để tìm kiếm Airbnb thực tế  
- Node.js/NPX - Để chạy máy chủ MCP bên ngoài  

Những gì bạn đã học được:  
- Tích hợp MCP: Kết nối các tác nhân Semantic Kernel với các máy chủ MCP bên ngoài  
- Truy cập dữ liệu thời gian thực: Tìm kiếm các tài sản Airbnb thực tế thông qua API trực tiếp  
- Giao tiếp qua giao thức: Sử dụng giao tiếp stdio giữa tác nhân và máy chủ MCP  
- Khám phá chức năng: Tự động phát hiện các chức năng có sẵn từ máy chủ MCP  
- Phản hồi theo luồng: Ghi lại và theo dõi các cuộc gọi chức năng trong thời gian thực  
- Kết xuất HTML: Định dạng phản hồi của tác nhân với các bảng được thiết kế và hiển thị tương tác  

Các bước tiếp theo:  
- Tích hợp thêm các máy chủ MCP (thời tiết, chuyến bay, nhà hàng)  
- Xây dựng hệ thống đa tác nhân kết hợp MCP và giao thức A2A  
- Tạo máy chủ MCP tùy chỉnh cho các nguồn dữ liệu của riêng bạn  
- Triển khai bộ nhớ hội thoại liên tục qua các phiên làm việc  
- Triển khai tác nhân lên Azure Functions với sự điều phối máy chủ MCP  
- Thêm khả năng xác thực người dùng và đặt chỗ  

Những lợi ích chính của kiến trúc MCP:  
- Chuẩn hóa: Giao thức phổ quát để kết nối các tác nhân AI với các công cụ bên ngoài  
- Dữ liệu thời gian thực: Truy cập thông tin trực tiếp, cập nhật từ nhiều dịch vụ khác nhau  
- Khả năng mở rộng: Dễ dàng tích hợp các nguồn dữ liệu và công cụ mới  
- Tương tác: Hoạt động trên các khung AI và nền tảng tác nhân khác nhau  
- Phân tách nhiệm vụ: Phân biệt rõ ràng giữa logic AI và truy cập dữ liệu bên ngoài  



---

**Tuyên bố miễn trừ trách nhiệm**:  
Tài liệu này đã được dịch bằng dịch vụ dịch thuật AI [Co-op Translator](https://github.com/Azure/co-op-translator). Mặc dù chúng tôi cố gắng đảm bảo độ chính xác, xin lưu ý rằng các bản dịch tự động có thể chứa lỗi hoặc không chính xác. Tài liệu gốc bằng ngôn ngữ bản địa nên được coi là nguồn thông tin chính thức. Đối với các thông tin quan trọng, khuyến nghị sử dụng dịch vụ dịch thuật chuyên nghiệp bởi con người. Chúng tôi không chịu trách nhiệm cho bất kỳ sự hiểu lầm hoặc diễn giải sai nào phát sinh từ việc sử dụng bản dịch này.
